## Mongodb的索引和如何在MongoEngine中使用索引

### 一、 mongodb的索引简介
[索引类型详解](https://mongoing.com/archives/26867), [mongodb中文网，文本索引](https://mongoing.com/docs/text-search.html)

- 单键索引, 联合索引  
- 除上述两种分类外，还有一些特殊的索引， 如:文本索引| 数组索引 | 稀疏索引 | 地理空间索引 | TTL索引等.

#### **索引的类型**

#### 单键索引

单键索引即索引的字段只有一个,是最基础的索引方式.  
在集合中使用`username`字段, `mongodb`创建一个单键索引,会自动将这个索引命名为 `username_1`

在索引优化的原则当中,有很重要的原则就是索引要建立在**基数高的的字段上**,所谓基数就是一个字段上不重复数值的个数,即我们在创建集合时年龄出现的数值是那么这个字段将会有100个不重复的数值,即字段的基数为100,而这个字段只会出现这个两个值,即字段的基础是2,这是一个相当低的基数,在这种情况下,索引的效率并不高并且会导致索引失效.

[索引基数](https://blog.csdn.net/mingyundezuoan/article/details/79038989)（cardinality）：索引中不重复的索引值的数量；
例如，某个数据列包含值1、3、7、4、7、3，那么它的基数就是4。
- 索引基数相对于数据表行数较高（也就是说，列中包含很多不同的值，重复的值很少）的时候，它的工作效果最好。
- 如果某数据列含有很多不同的年龄，索引会很快地分辨数据行。
- 如果某个数据列用于记录性别（只有”M”和”F”两种值），那么索引的用处就不大。
- 如果值出现的几率几乎相等，那么无论搜索哪个值都可能得到一半的数据行。在这些情况下，最好根本不要使用索引，因为查询优化器发现某个值出现在表的数据行中的百分比很高的时候，它一般会忽略索引，进行全表扫描。惯用的百分比界线是”30%”。


#### 联合索引

联合索引即索引上会有多个字段,例如使用`age`和 `sex` 两个字段创建一个索引, `{age_1_sex_1}`

#### 联合数组 和 联合数组索引

- 数组索引相比于其它索引来说索引条目和体积必然呈倍数增加,例如平均每个文档的数组的为10,那么这个集合的数组索引的条目数量将是普通索引的10倍.

- 联合数组索引就是含有数组字段的联合索引,这种索引不支持一个索引中含有多个数组字段,即一个索引中最多能有一个数组字段,这是为了避免索引条目爆炸式增长,假设一个索引中有两个数组字段,那么这个索引条目的数量将是普通索引的n*m倍

#### 文本索引（中文无法使用）  
文本索引将建立索引的文档字段先进行分词再进行检索,但是**目前还不支持中文分词**. [官网-社区版文本索引支持列表](https://www.mongodb.com/docs/manual/reference/text-search-languages/#std-label-text-search-languages)

#### 哈希索引(无分片，不使用)

为了**支持基于哈希的分片**，MongoDB提供了哈希索引类型，索引字段值的哈希值。这些索引在其范围内具有更随机的值分布，但仅 支持等值匹配且不支持范围查询。

#### **索引的属性**

#### 稀疏索引
**一般的索引会根据某个字段为整个集合创建一个索引**,即使某个文档不存这个字段,那么这个索引会把这个文档的这个字段当作 `null` 建立在索引当中.  
**稀疏索引不会对文档中不存在的字段建立索引**,如果这个字段存在但是为`null`时,则会创建索引`null`.

#### 唯一索引  
唯一索引就是在建立索引的字段上不能出现重复元素,除了单字段唯一索引还有联合唯一索引以及数组唯一索引(即数组之间不能有元素交集 )

#### 部分索引   *3.2版本的新功能。*
部分索引仅索引符合特定的过滤表达式的集合中的文档。通过索引集合中的文档子集，部分索引具有较低的存储要求，减少索引创建和维护的性能成本

### 二、 MongoEngine 索引简介 [官方文档](https://mongoengine-odm.readthedocs.io/guide/defining-documents.html#indexes)

You can specify indexes on collections to **make querying faster**. This is done by creating a list of index specifications called `indexes` in the `meta` dictionary, where an index specification may either be a single field name, a tuple containing multiple field names, or a dictionary containing a full index definition.

因此，索引创建的目的是加快查询速度。

A direction may be specified on fields by **prefixing the field name with a + (for ascending) or a - sign (for descending)**. Note that direction only **matters on compound indexes**. 
Text indexes may be specified by prefixing the field name with a $. 
Hashed indexes may be specified by prefixing the field name with a #:

复合索引(compound indexes)中, 在字段前加 `+`或 `-` 能够改变其中索引的排序方向。

```python
class Page(Document):
    category = IntField()
    title = StringField()
    rating = StringField()
    created = DateTimeField()
    meta = {
        'indexes': [
            'title',   # single-field index  # 注意建立在基数高的字段上
            '$title',  # text index  # 文本索引, 不支持中文
            '#title',  # hashed index  # 字段不完整时，注意使用时不会索引到不存在此字段的文档(无分片，不使用)
            ('title', '-rating'),  # compound index
            ('category', '_cls'),  # compound index
            {
                'fields': ['created'],
                'expireAfterSeconds': 3600  # ttl index
            }
        ]
    }
```

### 三、 单条索引详细的参数

(当向`meta = {'indexes': []}`的列表中传入字典时， 可以一并建立关于此项索引字段的其他设置。)  

If a dictionary is passed then additional options become available. Valid options include, but are not limited to:

    fields (Default: None) -> 索引字段
    The fields to index. Specified in the same format as described above.

    cls (Default: True) -> 为继承文档时，在索引前加入 类名(即不同级别的文档名)
    If you have polymorphic models that inherit and have allow_inheritance turned on, you can configure whether the index should have the _cls field added automatically to the start of the index.

    sparse (Default: False) -> 本索引是否设置为稀疏索引
    Whether the index should be sparse.

    unique (Default: False) -> 本索引是否为唯一索引
    Whether the index should be unique.

    expireAfterSeconds (Optional) -> 允许您通过以秒为单位设置使字段过期的时间来自动使集合中的数据过期。
    Allows you to automatically expire data from a collection by setting the time in seconds to expire the a field.

    name (Optional) -> 允许你指定索引的名称
    Allows you to specify a name for the index

    collation (Optional) -> 允许创建不区分大小写的索引
    Allows to create case insensitive indexes (MongoDB v3.4+ only)


### 四、 全局索引默认参数

There are a few top level defaults for all indexes that can be set:

```python
class Page(Document):
    title = StringField()
    rating = StringField()
    meta = {
        'index_opts': {},
        'index_background': True,
        'index_cls': False,
        'auto_create_index': True,
        'auto_create_index_on_save': False,
    }
```



- index_opts (Optional)  -> 设置任何默认索引选项  
Set any default index options - see the full options list

- index_background (Optional) -> 设置索引是否应在后台编制索引的默认值(mongodb: 建索引过程会阻塞其它数据库操作，background可指定以后台方式创建索引，即增加 "background" 可选参数。 "background" 默认值为false。)  
Set the default value for if an index should be indexed in the background

- index_cls (Optional) -> 一种为 `_cls` 关闭特定索引的方法。  
A way to turn off a specific index for _cls.

- auto_create_index (Optional) -> 当这是 True （默认）时，MongoEngine 将确保在首次使用 Document 时 MongoDB 中存在正确的索引。这可以在索引单独管理的系统中禁用。**禁用此功能将提高性能**。??  
When this is True (default), MongoEngine will ensure that the correct indexes exist in MongoDB when the Document is first used. This can be disabled in systems where indexes are managed separately. Disabling this will improve performance.

- auto_create_index_on_save (Optional) -> 当这是 True 时，MongoEngine 将确保每次运行时 MongoDB 中存在正确的索引。**启用它会降低性能**。默认值为`False`。  
When this is True, MongoEngine will ensure that the correct indexes exist in MongoDB each time save() is run. Enabling this will degrade performance. The default is False. This option was added in version 0.25.

如果从性能考虑， `auto_create_index` 和 `auto_create_index_on_save` 都将被禁用。

### 五、 复合索引和其他类型索引

Compound Indexes and Indexing sub documents -> 针对字典，嵌入式子文档中内容建立的索引 
Compound indexes can be created by adding the Embedded field or dictionary field name to the index definition.  
Sometimes its more efficient to index parts of Embedded / dictionary fields, in this case use ‘dot’ notation to identify the value to index eg: rank.title

#### 其他索引  
[地理空间索引 Geospatial indexes](https://mongoengine-odm.readthedocs.io/guide/defining-documents.html#geospatial-indexes),  
[生存时间 (TTL)索引 Time To Live (TTL) indexes](https://mongoengine-odm.readthedocs.io/guide/defining-documents.html#time-to-live-ttl-indexes),  
[比较索引 Comparing Indexes](https://mongoengine-odm.readthedocs.io/guide/defining-documents.html#comparing-indexes)：用`mongoengine.Document.compare_indexes() `将数据库中的 *实际索引* 与 *文档定义* 的索引进行比较。这对于**维护目的很有用**，并确保您拥有适合您的架构的正确索引。

### 六、 索引练习

实际上比较常用的索引是 单键索引，联合数组， 联合索引，混合索引； 单个索引参数和全局索引参数使用默认项目即可.

In [3]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

#### 一般索引的创建

In [5]:
# 单键索引，组合索引
class BookWithIndex(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3.5")
    author = StringField()
    meta = {
        "db_alias": TEST_DB1,
        "indexes": [
            "bookid",
            ("bookid", "+name")
        ]
    }

In [6]:
BookWithIndex(bookid="2idjs333", name="四书", author="孔子").save()

<BookWithIndex: BookWithIndex object>

#### 查看某个已建立文档的索引

In [10]:
BookWithIndex.list_indexes()

[[('bookid', 1)], [('bookid', 1), ('name', 1)], [('_id', 1)]]

#### 复合索引的建立

In [17]:
from mongoengine import EmbeddedDocument, EmbeddedDocumentField
class BookAuthor(EmbeddedDocument):
    name = StringField()
    age = StringField()
    meta = {"db_alias": TEST_DB1}
# 更复杂的索引 -> 待补充
class BookWithCompoundIndex(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3.5")
    author = EmbeddedDocumentField(BookAuthor)
    meta = {
        "db_alias": TEST_DB1,
        "indexes": [
            "author.name",
        ]
    }


In [18]:
aut = BookAuthor(name="孔子", age="春秋")
BookWithCompoundIndex(bookid="2idjs333", name="四书", author=aut).save()

<BookWithCompoundIndex: BookWithCompoundIndex object>

In [19]:
BookWithCompoundIndex.list_indexes()

[[('author.name', 1)], [('_id', 1)]]

### 七、 总结

索引可以使用mongoengine建立，也可以使用mongodb shell或者界面软件建立。他们所基于的查询和管理方式都是基于mongodb数据库系统, 可以选择便于管理的方法建立和使用索引。